In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
from process_raw_data import *

# 6060 term

In [3]:
import pandas as pd
df = pd.read_csv("raw_data/6060_terminology_glossary.csv")
df = df[['english', 'chinese', 'arabic', 'french', 'russian', 'japanese']]

In [4]:
# !python -m spacy download en_core_web_trf

In [5]:
import spacy
nlp = spacy.load("en_core_web_trf")
import en_core_web_trf
nlp = en_core_web_trf.load()


In [6]:
word_lst = df['english'].values.tolist()
pos_lst = []
for word in word_lst:
    doc = nlp(word)
    for token in doc:
        pos_lst.append(token.pos_)
        # print(token.pos_)
        # ADJ, NOUN, PROPN, VERB
len(pos_lst), len(word_lst)

(309, 309)

In [7]:
df['pos_tag'] = pos_lst
df['pos_tag'].value_counts()

pos_tag
NOUN     155
PROPN     68
VERB      37
ADJ       35
ADV       12
ADP        2
Name: count, dtype: int64

In [8]:
df.loc[df['english'] == 'question']

,english,chinese,arabic,french,russian,japanese,pos_tag
60,question,问题,سؤال,question,вопрос,質問,NOUN


In [9]:
df.loc[df['pos_tag'] == 'NOUN', 'english'].values.tolist()

['Modeling',
 'task',
 'dataset',
 'models',
 'words',
 'language',
 'languages',
 'grammar',
 'linguistics',
 'source',
 'vocabulary',
 'tasks',
 'text',
 'speech',
 'detection',
 'sentence',
 'spans',
 'texts',
 'word',
 'model',
 'news',
 'training',
 'tokens',
 'encoding',
 'encode',
 'token',
 'number',
 'features',
 'transformer',
 'transformers',
 'metrics',
 'question',
 'embeddings',
 'information',
 'resources',
 'analysis',
 'paper',
 'Morphology',
 'research',
 'tokenization',
 'algorithm',
 'subword',
 'morphemes',
 'representation',
 'algorithms',
 'compositionality',
 'input',
 'document',
 'verbs',
 'encoder',
 'vectors',
 'nouns',
 'pronouns',
 'inference',
 'decoding',
 'correlations',
 'attention',
 'computation',
 'prediction',
 'classification',
 'problem',
 'vector',
 'baselines',
 'gradient',
 'sequences',
 'data',
 'quality',
 'finetuning',
 'user',
 'evaluation',
 'translations',
 'translation',
 'regression',
 'ablation',
 'method',
 'answer',
 'relations',
 '

In [10]:
df = df.loc[~df['english'].isin(unrelated_terms['6060_initiative'])]

In [11]:
df.loc[df['english'].isin(to_lowercase_terms['6060_initiative']), 'english'] = df.loc[df['english'].isin(to_lowercase_terms['6060_initiative']), 'english'].apply(lambda x: x.lower())

In [12]:
df

,english,chinese,arabic,french,russian,japanese,pos_tag
1,modeling,造型,نمذجة,modélisation,моделирование,モデリング,NOUN
2,lexical,词汇,معجمي,lexicale,словарный,語彙的,ADJ
3,task,任务,مهمة,tâche,Задача,タスク,NOUN
4,dataset,数据集,مجموعة بيانات,données,набор данных,データセット,NOUN
5,models,模型,نماذج,modèles,Модели,モデル,NOUN
...,...,...,...,...,...,...,...
304,extraction,提取,استخراج,extraction,Извлечение,抽出,NOUN
305,XLNet,XLNet,XLNet,XLNet,XLNet,XLNet,PROPN
306,multitask,多任务,متعددة المهام,multi-tâches,Многозадачность,マルチタスク,PROPN
307,MTDNN,MTDNN,MTDNN,MTDNN,MTDNN,MTDNN,PROPN


In [14]:
def align_capitalization(w1, w2):
    w3 = ""
    if w1[0].isupper():
        w3 += w2[0].upper()
    else:
        w3 += w2[0].lower()
    w3 += w2[1:]
    return w3
df['french'] = df.apply(lambda row: align_capitalization(row['english'], row['french']), axis=1)
df['russian'] = df.apply(lambda row: align_capitalization(row['english'], row['russian']), axis=1)

In [22]:
df.to_csv("processed_data/6060_terminology_glossary.csv")

# wiki_ai & wiki_cs

In [41]:
import json
wiki_ai = json.load(open("raw_data/wiki_ai.json", 'r'))
wiki_cs = json.load(open("raw_data/wiki_cs.json", 'r'))

In [42]:
wiki_ai = [{"term": format_wiki_terms(item['term']), "explanation": item['explanation']} for item in wiki_ai]
wiki_cs = [{"term": format_wiki_terms(item['term']), "explanation": item['explanation']} for item in wiki_cs]

In [43]:
wiki_ai[0]

{'term': 'abductive logic programming',
 'explanation': 'A high-level knowledge-representation framework that can be used to solve problems declaratively based on abductive reasoning . It extends normal logic programming by allowing some predicates to be incompletely defined, declared as abducible predicates.'}

In [44]:
# translate the English term to other languages
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append("../translator")
sys.path.append("..")
from translate_term import translate_term
df = pd.DataFrame().from_records(wiki_ai)
for language in tqdm(["Arabic", "French", "Japanese", "Russian"]):
    translated_term = []
    for item in tqdm(wiki_ai):
        translated_term.append(translate_term(item['term'], language, explanation=item['explanation'], model='gpt-4o'))
    df[f"term_{language.lower()}"] = translated_term

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [11:16<00:00, 169.23s/it]


In [45]:
df.to_csv("processed_data/wiki_ai2.csv")

In [46]:
# translate the English term to other languages
from tqdm import tqdm
import pandas as pd
import sys
sys.path.append("../translator")
sys.path.append("..")
from translate_term import translate_term
df = pd.DataFrame().from_records(wiki_cs)
for language in tqdm(["Chinese", "Arabic", "French", "Japanese", "Russian"]):
    translated_term = []
    for item in tqdm(wiki_cs):
        translated_term.append(translate_term(item['term'], language, explanation=item['explanation'], model='gpt-4o'))
    df[f"term_{language.lower()}"] = translated_term

100%|██████████| 5/5 [11:24<00:00, 136.85s/it]


In [47]:
df.to_csv("processed_data/wiki_cs.csv")

# aggregate term dictionary

In [60]:
wiki_ai = pd.read_csv("processed_data/wiki_ai.csv")
wiki_cs = pd.read_csv("processed_data/wiki_cs.csv")
initiative = pd.read_csv("processed_data/6060_terminology_glossary.csv")

In [61]:
df = pd.DataFrame()
langs = ['english', 'chinese', 'arabic', 'french', 'russian', 'japanese']
for lang in langs:
    lst = []
    lst.extend(initiative[lang].values.tolist())
    if lang == 'english':
        lst.extend(wiki_ai['term'].values.tolist())
        lst.extend(wiki_cs['term'].values.tolist())
    else:
        lst.extend(wiki_ai[f'term_{lang}'].values.tolist())
        lst.extend(wiki_cs[f'term_{lang}'].values.tolist())
    df[lang] = lst

In [62]:
df.drop_duplicates(inplace=True)

In [63]:
# check non-unique values
non_unique_vals = df['english'][df['english'].duplicated(keep=False)]
non_unique_rows = df[df['english'].isin(non_unique_vals)]
non_unique_rows

,english,chinese,arabic,french,russian,japanese
41,transformer,转换器,محول,conversion,преобразователь,トランスフォーマー
60,algorithm,算法,خوارزمية,algorithme,алгоритм,アリゴリズム
86,computation,计算,حساب,calcul,вычисление,計算
108,method,方法,طريقة,méthode,метод,方法
111,sequence,顺序,التسلسل,séquence,последовательность,シーケンス
122,parameter,参数,معلمة,paramètres,параметр,パラメータ
129,graph,图形,رسم بياني,"graphique, diagramme",диаграммa,グラフ
133,event,事件,حدث,activité,событие,イベント
140,heuristic,启发式,إرشادي,heuristique,эвристический,ヒューリスティック
158,syntax,句法,بناء جملة,syntaxe,синтаксис,構文論


In [64]:
terms = list(set(non_unique_rows['english'].values.tolist()))
len(terms)

29

In [65]:
terms

['abstract data type',
 'algorithm',
 'syntax',
 'computation',
 'computer vision',
 'robotics',
 'semantics',
 'machine vision',
 'concatenation',
 'heuristic',
 'cognitive science',
 'computer science',
 'node',
 'abstraction',
 'software',
 'data mining',
 'parameter',
 'event',
 'logic programming',
 'intelligent agent',
 'agent architecture',
 'machine learning',
 'open-source software',
 'sequence',
 'method',
 'domain',
 'transformer',
 'internet',
 'graph']

In [ ]:
df.to_csv("processed_data/merged_term_dict.csv")

In [ ]:
# next step: manually unify duplicated rows that are leftover